# Chapter 14 - Layering and Clean Architecture

- Efficient way to partition and organize units of logic i.e. MVC
- Logical organization of code
- Horizontal slicing
- Classic layering
    * Presenstion layer
    * Domain layer (business logic)
    * Data layer or persistance layer
- Sometimes layers are divided among teams, leaving knowledge gaps
- Splitting layers veritically and horizontally
- Assemblies are consumable units of compiled code i.e. `.dll` or `.exe`
    * Can be deployed as nuget packages

## Domain layer

- Rich vs anemic

### Rich domain model

- Operations in the domain model classes

### Anemic domain model

- Logic goes into a service layer

## Service Layer

- Service layer can be just a sub-folder named Services \
- It could be seperate assemblies for abstractions and implementations
- It could be its own tier i.e. REST api
- Used with Anemic domain models

## Data layer

- Common patterns are unit of work and Repository patterns
- Modern data layers leverage ORM such as entity framework
    * implements the above patterns for us
- EF core is lighter weight and faster than EF 6
    * Not the same thing
- Can use Dapr for complete control of your sql code

### Repository pattern

- Not needed if using EF core

Example interface

```csharp
public interface IRepository<T, TId> where T : class, IEntity<TId>
{
    Task<IEnumerable<T>> AllAsync(CancellationToken cancellationToken);
    Task<T?> GetByIdAsync(TId id, CancellationToken cancellationToken);
    Task<T> CreateAsync(T entity, CancellationToken cancellationToken);
    Task UpdateAsync(T entity, CancellationToken cancellationToken);
    Task DeleteAsync(TId id, CancellationToken cancellationToken);
}
public interface IEntity<TId>
{
    TId Id { get; }
}
```

### Unit of Work

- Allows us to combine multiple changes into a single transaction (Single database call)
- IMplemented as `DbContext` in EF


In [11]:
classDiagram
    direction LR
    namespace Presentation {
        class Program
    }
    namespace Domain {
        class IProductService
        class ProductService
    }
    namespace Data.Abstract {
        class IProductRepository
    }
    namespace Data.EF {
        class ProductRepository
    }

## Project

See solution in `C14\AbstractLayer`

- Seperate models for each layer

### Data.Abstract Layer

In [12]:
using System.Collections.Generic;
using System.Threading;

public class Product
{
    public int? Id { get; set; }
    public string? Name { get; set; }
    public int QuantityInStock { get; set; }
}

public interface IProductRepository
{
    Task<IEnumerable<Product>> AllAsync(CancellationToken cancellationToken);
    Task<Product?> FindByIdAsync(int productId, CancellationToken cancellationToken);
    Task CreateAsync(Product product, CancellationToken cancellationToken);
    Task UpdateAsync(Product product, CancellationToken cancellationToken);
    Task DeleteAsync(int productId, CancellationToken cancellationToken);
}

### Data.EF Layer

In [13]:
#r "nuget: Microsoft.EntityFrameworkCore"
#r "nuget: Microsoft.EntityFrameworkCore.InMemory"

Installed Packages Microsoft.EntityFrameworkCore, 9.0.6 Microsoft.EntityFrameworkCore.InMemory, 9.0.6

In [14]:
//using Data.Abstract;
using Microsoft.EntityFrameworkCore;

public class ProductContext : DbContext
{
    public ProductContext(DbContextOptions options)
        : base(options)
    {
    }

    public DbSet<Product> Products => Set<Product>();
}

public class ProductNotFoundException : Exception
{
    public ProductNotFoundException(int productId)
        : base($"The product '{productId}' was not found.")
    {
        ProductId = productId;
    }

    public int ProductId { get; }
}

public class ProductRepository : IProductRepository
{
    private readonly ProductContext _db;
    public ProductRepository(ProductContext db)
    {
        _db = db ?? throw new ArgumentNullException(nameof(db));
    }

    public async Task<IEnumerable<Product>> AllAsync(CancellationToken cancellationToken)
    {
        return await _db.Products.ToArrayAsync(cancellationToken);
    }

    public async Task CreateAsync(Product product, CancellationToken cancellationToken)
    {
        _db.Products.Add(product);
        await _db.SaveChangesAsync(cancellationToken);
    }

    public async Task DeleteAsync(int productId, CancellationToken cancellationToken)
    {
        var product = await FindByIdAsync(productId, cancellationToken);
        if (product == null)
        {
            throw new ProductNotFoundException(productId);
        }
        _db.Products.Remove(product);
        await _db.SaveChangesAsync(cancellationToken);
    }

    public async Task<Product?> FindByIdAsync(int productId, CancellationToken cancellationToken)
    {
        var product = await _db.Products.FindAsync(new object[] { productId }, cancellationToken);
        return product;
    }

    public async Task UpdateAsync(Product product, CancellationToken cancellationToken)
    {
        _db.Entry(product).State = EntityState.Modified;
        await _db.SaveChangesAsync(cancellationToken);
    }
}


### Domain layer

In [15]:
// public class Product
// {
//     public int? Id { get; set; }
//     public string? Name { get; set; }
//     public int QuantityInStock { get; set; }
// }

public class NotEnoughStockException : Exception
{
    public NotEnoughStockException(int quantityInStock, int amountToRemove)
        : base($"You cannot remove {amountToRemove} item(s) when there is only {quantityInStock} item(s) left.")
    {
        QuantityInStock = quantityInStock;
        AmountToRemove = amountToRemove;
    }

    public int QuantityInStock { get; }
    public int AmountToRemove { get; }
}

//Services

public interface IProductService
{
    Task<IEnumerable<Product>> AllAsync(CancellationToken cancellationToken);
}

public interface IStockService
{
    Task<int> AddStockAsync(int productId, int amount, CancellationToken cancellationToken);
    Task<int> RemoveStockAsync(int productId, int amount, CancellationToken cancellationToken);
}

public class ProductNotFoundException : Exception
{
    public ProductNotFoundException(int productId)
        : base($"The product '{productId}' was not found.")
    {
        ProductId = productId;
    }

    public int ProductId { get; }
}

public class ProductService : IProductService
{
    private readonly IProductRepository _repository;
    public ProductService(IProductRepository repository)
    {
        _repository = repository ?? throw new ArgumentNullException(nameof(repository));
    }

    public async Task<IEnumerable<Product>> AllAsync(CancellationToken cancellationToken)
    {
        return (await _repository.AllAsync(cancellationToken)).Select(p => new Product
        {
            Id = p.Id,
            Name = p.Name,
            QuantityInStock = p.QuantityInStock
        });
    }
}

public class StockService : IStockService
{
    private readonly IProductRepository _repository;
    public StockService(IProductRepository repository)
    {
        _repository = repository ?? throw new ArgumentNullException(nameof(repository));
    }

    public async Task<int> AddStockAsync(int productId, int amount, CancellationToken cancellationToken)
    {
        var product = await _repository.FindByIdAsync(productId, cancellationToken);
        if (product == null)
        {
            throw new ProductNotFoundException(productId);
        }
        product.QuantityInStock += amount;
        await _repository.UpdateAsync(product, cancellationToken);

        return product.QuantityInStock;
    }

    public async Task<int> RemoveStockAsync(int productId, int amount, CancellationToken cancellationToken)
    {
        var product = await _repository.FindByIdAsync(productId, cancellationToken);
        if (product == null)
        {
            throw new ProductNotFoundException(productId);
        }
        if (amount > product.QuantityInStock)
        {
            throw new NotEnoughStockException(product.QuantityInStock, amount);
        }
        product.QuantityInStock -= amount;
        await _repository.UpdateAsync(product, cancellationToken);

        return product.QuantityInStock;
    }
}


### Presentation layer

In [16]:
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.Extensions.Hosting.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Mvc.ViewFeatures.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Diagnostics.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Http.dll"
#r "C:\Program Files\dotnet\shared\Microsoft.AspNetCore.App\9.0.2\Microsoft.AspNetCore.Http.Results.dll"

In [17]:
string[] args = {"--urls","http://localhost:7002"};

In [18]:
//using Domain.Services;
using Microsoft.AspNetCore.Http;
using Microsoft.EntityFrameworkCore;
using Microsoft.EntityFrameworkCore.Diagnostics;
using Microsoft.AspNetCore.Http.HttpResults;
using Microsoft.AspNetCore.Routing;
using Microsoft.AspNetCore.Builder;
using Microsoft.Extensions.DependencyInjection;

var builder = WebApplication.CreateBuilder(args);
builder.Services
    // Domain Layer
    .AddScoped<IProductService, ProductService>()
    .AddScoped<IStockService, StockService>()

    // Data Layer (mapping Data.Abstract to Data.EF)
    .AddScoped<IProductRepository, ProductRepository>()
    .AddDbContext<ProductContext>(options => options
        .UseInMemoryDatabase("ProductContextMemoryDB")
        .ConfigureWarnings(builder => builder.Ignore(InMemoryEventId.TransactionIgnoredWarning))
    )
;

var app = builder.Build();

app.MapGet("/products", async (IProductService productService, CancellationToken cancellationToken)
    => (await productService.AllAsync(cancellationToken)).Select(p => new {
        p.Id,
        p.Name,
        p.QuantityInStock
    }));
app.MapPost("/products/{productId:int}/add-stocks", async (int productId, AddStocksCommand command, IStockService stockService, CancellationToken cancellationToken) =>
{
    var quantityInStock = await stockService.AddStockAsync(productId, command.Amount, cancellationToken);
    return new StockLevel(quantityInStock);
});
app.MapPost("/products/{productId:int}/remove-stocks", async (int productId, RemoveStocksCommand command, IStockService stockService, CancellationToken cancellationToken) =>
{
    try
    {
        var quantityInStock = await stockService.RemoveStockAsync(productId, command.Amount, cancellationToken);
        var stockLevel = new StockLevel(quantityInStock);
        return Results.Ok(stockLevel);
    }
    catch (NotEnoughStockException ex)
    {
        return Results.Conflict(new
        {
            ex.Message,
            ex.AmountToRemove,
            ex.QuantityInStock
        });
    }
});

using (var seedScope = app.Services.CreateScope())
{
    var db = seedScope.ServiceProvider.GetRequiredService<ProductContext>();
    await ProductSeeder.SeedAsync(db);
}
app.RunAsync();

public class AddStocksCommand
{
    public int Amount { get; set; }
}

public class RemoveStocksCommand
{
    public int Amount { get; set; }
}

public class StockLevel
{
    public StockLevel(int quantityInStock)
    {
        QuantityInStock = quantityInStock;
    }

    public int QuantityInStock { get; set; }
}

public static class ProductSeeder
{
    public static Task SeedAsync(ProductContext db)
    {
        db.Products.Add(new Product
        {
            Id = 1,
            Name = "Banana",
            QuantityInStock = 50
        });
        db.Products.Add(new Product
        {
            Id = 2,
            Name = "Apple",
            QuantityInStock = 20
        });
        db.Products.Add(new Product
        {
            Id = 3,
            Name = "Habanero Pepper",
            QuantityInStock = 10
        });
        return db.SaveChangesAsync();
    }
}

info: Microsoft.EntityFrameworkCore.Update[30100]
      Saved 3 entities to in-memory store.
info: Microsoft.Hosting.Lifetime[14]
      Now listening on: http://localhost:7002
info: Microsoft.Hosting.Lifetime[0]
      Application started. Press Ctrl+C to shut down.
info: Microsoft.Hosting.Lifetime[0]
      Hosting environment: Production
info: Microsoft.Hosting.Lifetime[0]
      Content root path: c:\Users\jason\training\dotnet\Architecting-ASP.NET-Core-Applications-3E\MyNotes


In [19]:
using System.Net.Http;

var httpClient = new HttpClient();

var response = await httpClient.GetAsync("http://localhost:7002/products");
response.EnsureSuccessStatusCode();
var content = await response.Content.ReadAsStringAsync();
content.DisplayAs("application/json")

In [20]:
app.StopAsync();

## Sharing the model

- Presentation layer should use View model or DTO objects
- See `C14\SharedModel` for example
- Be careful as it can get messy

## Clean Architecture

- UI, Core, and Infrastructure
- Core
    * Domain models and entities
    * Domain services
- Infrastructure
    * UIs
    * Data access
    * Infrastructure related services
- These layers can be split into smaller layers
- Dependencies can only point inward

### Infrastructure

- UIs
- Implementations
    * i.e. REST calls, DB calls, disk access, etc

### Core

- Use cases
    - Interfaces
    - Domain services   
- Entities (Models)

### Project - `C14\CleanArchitecture`

## Real life

- Domain-heavy and logic intesive project will benefit from a domain layer
- Data management project maybe only need a presentation and data layer
- It depends on the project
    * Skill level of the team
    * Requirments
    * Budget constraints
    * etc
- Dont over-engineer
- YAGNI
- Educate customers and clients on expectations
- Educate peers
- Utilize tools for simple data-driven applications like no code
- Keep it as simple as possible. Only as much complexity as needed
- Consider who will be supporting it


## Resources

- [Part 5: Repositories, the ClanRepository, and integration testing](https://www.forevolve.com/en/articles/2017/08/25/design-patterns-web-api-service-and-repository-part-5/)
- [EF - Using Transactions](https://learn.microsoft.com/en-us/ef/core/saving/transactions)
- [MS Learn - Architecture](https://learn.microsoft.com/en-us/dotnet/architecture/modern-web-apps-azure/common-web-application-architectures)
-[GitHub - ardalis/CleanArchitecture](https://github.com/ardalis/CleanArchitecture)
-[GitHub - jasontaylordev/CleanArchitecture](https://github.com/jasontaylordev/CleanArchitecture)

